# a) ineær regresjon i 3 dimensjoner:
<!-- Based on [linear-2d](https://gitlab.com/ntnu-tdat3025/regression/linear-2d) by Ole Christian Eidheim. -->

In [1]:
import torch
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d, art3d
from matplotlib.animation import FuncAnimation
%matplotlib qt


global x_train
global y_train
global model
global ax

## Import data from csv

In [2]:
train = pd.read_csv('day_length_weight.csv')

day = train.iloc[0::,0]
day_list = day.values.tolist()
x1_train = torch.tensor(day_list).reshape(-1, 1)

length = train.iloc[0::,1]
length_list = length.values.tolist()
x2_train = torch.tensor(length_list).reshape(-1, 1)

weight = train.iloc[0::,2]
weight_list = weight.values.tolist()
y_train = torch.tensor(weight_list).reshape(-1, 1)

x_train = torch.cat((x1_train, x2_train),1).float()

print(f"{x_train[0:10] = }",f"\n{y_train[0:10] = }")

x_train[0:10] = tensor([[ 604.0000,   84.8655],
        [ 964.0000,   92.8221],
        [1640.0000,   98.5218],
        [ 642.0000,   85.6332],
        [ 467.0000,   75.2944],
        [ 447.0000,   79.7825],
        [1450.0000,   99.4479],
        [1472.0000,  105.3180],
        [1661.0000,  116.2802],
        [1776.0000,  107.2823]]) 
y_train[0:10] = tensor([[13.0302],
        [14.7191],
        [21.0195],
        [12.7910],
        [10.5395],
        [ 9.3395],
        [12.4751],
        [12.1663],
        [20.8239],
        [16.3704]])


## Define Linear Regression Model class

In [3]:
class LinearRegressionModel:
    def __init__(self):
        # Model variables
        self.W = torch.tensor([[-0.2], [0.53]], requires_grad=True)  # requires_grad enables calculation of gradients
        self.b = torch.tensor([[3.1]], requires_grad=True)

    # Predictor
    def f(self, x):
        return x @ self.W + self.b  # @ corresponds to matrix multiplication

    # Uses Mean Squared Error
    def loss(self, x, y):
        return torch.nn.functional.mse_loss(self.f(x), y)

## Optimize with stochastic gradient descent (SGD)
PyTorch's built in optimizer does a lot of the work for us.

In [4]:
model = LinearRegressionModel()

# set variables
scale = 10**(-2)
epoch_count = 1
step_length = 0.008

# downscale
x_train = x_train * scale
y_train = y_train * scale

# Optimize: adjust W and b to minimize loss using stochastic gradient descent
optimizer = torch.optim.SGD([model.W, model.b], step_length)

for epoch in range(epoch_count):
    
    model.loss(x_train, y_train).backward()  # Compute loss gradients

    optimizer.step()  # Perform optimization by adjusting W and b,

    optimizer.zero_grad()  # Clear gradients for next step

def run_epoch(plot, epo):
    model.loss(x_train, y_train).backward()  # Compute loss gradients

    optimizer.step()  # Perform optimization by adjusting W and b,

    optimizer.zero_grad()  # Clear gradients for next step
    plot.set_title("\nW = %s\n b = %s\n loss = %s\n epoch = %s" % ([round(model.W[0].item(),4), round(model.W[1].item(),4)] ,round(model.b.item(),4), round(model.loss(x_train, y_train).item(),4), epo-100))
    

W = [-0.20000000298023224, 0.5299999713897705], b = 3.0999999046325684, loss = 3.636127233505249


## Visualize data

In [5]:
# Create the figure
fig = plt.figure()

# Add an axes
ax = fig.add_subplot(111,projection='3d')

def animate(i):

    fig.clear()

    ax = fig.add_subplot(111,projection='3d')

    run_epoch(ax, i)

    ax.view_init(20, 50 + ((i/5) % 360))

    # max min values for gird
    x = torch.stack([torch.min(x_train, 0).values, torch.max(x_train, 0).values])

    # setup grid vars
    x1_grid, x2_grid = np.meshgrid(np.linspace(x[0,0], x[1,0], 10), np.linspace(x[0,1], x[1,1], 10))
    y_grid = np.empty([10, 10])


    # set grid values using model.f()
    for i in range(0, x1_grid.shape[0]):
        for j in range(0, x1_grid.shape[1]):
            
            # Xᵢ
            x = torch.tensor( [[ x1_grid[i, j], x2_grid[i, j] ]] ).float()
            
            # yᵢ = f(xᵢ)
            y_grid[i, j] = model.f(x)

    # plot wireframe
    ax.plot_wireframe(x1_grid, x2_grid, y_grid, color='orange')

    # and plot the points 
    ax.scatter(x_train[:,0],x_train[:,1],y_train,  color='blue')


ani = FuncAnimation(plt.gcf(), animate, interval=1)

plt.show()


In [6]:
# Print model variables and loss
print("W = %s, b = %s, loss = %s" % (model.W, model.b/scale, model.loss(x_train, y_train)))

W = tensor([[-0.2211],
        [ 0.5107]], requires_grad=True), b = tensor([[307.1173]], grad_fn=<DivBackward0>), loss = tensor(3.1739, grad_fn=<MseLossBackward>)
